# 실제 로직 테스트 코드

## 1. 환경 설정

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 사용할 클래스 목록
from dataclasses import dataclass

## 검색될 내용
@dataclass
class SearchResult:
  keyword: str
  title: str
  link: str
  content: str

## VDB에 들어갈 메타 데이터
@dataclass
class Metadata:
  keyword: str
  link: str

In [3]:
from langchain_upstage import ChatUpstage
from langchain.schema import SystemMessage, HumanMessage

# 답변 모델
llm_main = ChatUpstage(
    model="solar-mini",
    temperature=0.7,
)

# 검증 모델
llm_validate = ChatUpstage(
    model="solar-pro2",
    temperature=0.7
)

---
## 2. 사전 요소 정의 
### 1. RAG 기능 쪽 변수 및 함수

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# 스플리터 정의
splitter = RecursiveCharacterTextSplitter(
  chunk_size=300,
  chunk_overlap=50
)

# 임베딩 모델 설정
vstore = Chroma(
  collection_name="knowledge_base",
  embedding_function=UpstageEmbeddings(model="embedding-query"),
  persist_directory="./chroma"
)

# 검색자
retriever=vstore.as_retriever(
  search_type="mmr",
  search_kwargs={
    "k":4,
    "fetch_k":20,
    "lambda_mult":0.5
  }
)

# 연관 되어있는 것으로 볼 범위
THRESHOLD = 1.55

In [5]:
from langchain.schema import Document

# VDB에 저장된 문서 묶음 검색해오기
def search_documents(keyword: str) -> list[Document]:
  scored = vstore.similarity_search_with_score(keyword, k=20)

  filtered =  [
    doc for doc, score in scored if score <= THRESHOLD
  ]
  return filtered

# 검색 후 프롬프트도 만들기
def get_prompt_on_retrieve(keyword: str, prompt: str) -> str:
  search_results = search_documents(keyword=keyword)

  # 프롬프트에 내용 삽입하기
  result = prompt
  for items in search_results:
     result += ("\n" + items.page_content)

  return result

# contents를 스플리터로 자르고 메타데이터 붙이기
def split_documents(contents: list[str], metadatas: list[Metadata]):
  docs: list[Document] = []

  for i in range(len(contents)):
    splits = splitter.create_documents(
      texts=[contents[i]],
      metadatas=[{
        "keyword":metadatas[i].keyword,
        "link":metadatas[i].link,
      }],
    )
    docs.extend(splits)
  return docs

# SearchResult로 가져온 문서들을 contents와 metadatas로 분리
def embed_documents(datas: list[SearchResult]):
  # 데이터를 content와 메타데이터로 분리
  contents: list[str] = [("# " + data.title + "\n" + data.content) for data in datas]
  metadatas: list[Metadata] = [
    Metadata(
      data.keyword,
      data.link
    ) for data in datas
  ]

  docs = split_documents(contents=contents, metadatas=metadatas)

  # 벡터 스토어에 데이터 저장
  vstore.add_documents(docs)
  
  return {"result":"complete"}

### 2. llm 모델 호출 함수

In [6]:
analyst_form="""
{
    "answer":"문서를 바탕으로 한 추론과 결론",
    "link":[
        "근거로 사용한 문서의 링크 1",
        "근거로 사용한 문서의 링크 2",
        ....
    ]
}
"""

def analyst_prompt(keyword: str, form: str):
    return f"""
당신은 최고의 검색어 분석 전문가 입니다.
입력으로 키워드와 여러 문서들의 내용이 주어집니다.
해당 문서들은 모두 {keyword}로 검색된 내용들 중 가장 최근의 기사들입니다. 
아래와 같은 규칙과 방법으로 현재 이 키워드가 왜 트랜디해졌는지 분석하십시오.

근거는 반드시 문서만을 참고하여 주십시오.
각 문서마다 핵심 이슈를 요약하여 유사한 문서가 있는지 확인해야합니다.
여러개의 문서 중 유사한 주제로 3개 이상 동시에 발견되어야합니다.
감정적 단어나 추측적 표현("~일 것이다")은 피하고, 실제 문서 근거를 기반으로 결론을 도출합니다.\

출력 형식은 다음과 같이 JSON으로 주십시오: {form}
"""

validator_form="""
{
    "validation": "yes" 또는 "no",
    "reason": "판단의 이유 한 두 줄로 설명"
}
"""

def validator_prompt(keyword: str, form: str):
    return f"""
당신은 엄격한 리뷰어 입니다.
이 내용은 벡터DB에서 {keyword}로 검색되었습니다.
실제로 해당 내용과 관련이 있는지 간단한 문구와 함께 검증하시오.

다음과 같은 양식으로 한 줄로 작성해야 합니다: {form}
"""

In [7]:
def analyst_llm(prompt: str, keyword: str) -> str:
    """
    메인 llm 호출 함수
    """
    messages = [
        SystemMessage(content=analyst_prompt(keyword, analyst_form)),
        HumanMessage(content=prompt)
    ]

    # invoke()는 ChatModel 표준 인터페이스입니다.
    result = llm_main.invoke(messages)

    # result는 보통 AIMessage 객체이며 .content에 텍스트가 들어 있습니다.
    return result.content.strip()

def validator_llm(prompt: str, keyword: str) -> str:
    """
    검증 llm 호출 함수
    """
    messages = [
        SystemMessage(content=validator_prompt(keyword, validator_form)),
        HumanMessage(content=prompt),
    ]

    # invoke()는 ChatModel 표준 인터페이스입니다.
    result = llm_validate.invoke(messages)

    # result는 보통 AIMessage 객체이며 .content에 텍스트가 들어 있습니다.
    return result.content.strip()

### 3. 크롤링 기능

In [8]:
import requests
from bs4 import BeautifulSoup
import json
import time
from urllib.parse import quote_plus

def get_keyword_news(keyword):
    href_links = []

    origin_url = f'https://search.naver.com/search.naver?where=news&query={keyword}'

    response = requests.get(origin_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    naver_spans = soup.find_all('span', string='네이버뉴스')
   
    for news in naver_spans:
        anchor_tag = news.find('a')
        if anchor_tag:
            href = anchor_tag.get('href')
            href_links.append(href)
    return href_links
    
def get_news_from_naver(keyword, urls) -> list[SearchResult]:
    results: list[SearchResult] = []
    title = ""
    content = ""
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        if 'entertain.naver.com' in url:
            title_tag = soup.find('div', class_='ArticleHead_article_head_title__YUNFf')
            content_tag = soup.find('div', class_='_article_content')
            title = title_tag.get_text(strip=True)
            content = content_tag.get_text(strip=True)
        
        elif 'n.news.naver.com' in url:
            title_tag = soup.find('div', class_='media_end_head_title')
            content_tag = soup.find('div', class_='newsct_article')
            title = title_tag.find('span').get_text(strip=True)
            content = content_tag.get_text(strip=True)
        elif 'm.sports.naver.com' in url:
            title_tag = soup.find('h2', class_='ArticleHead_article_title__qh8GV')
            content_tag = soup.find('div', class_='ArticleContent_comp_article_content__luOFM')
            title = title_tag.get_text(strip=True)
            content = content_tag.get_text(strip=True)
        else:
            print("Unsupported URL format:", url)

        results.append(SearchResult(
            keyword=keyword,
            link=url,
            title=title,
            content=content
        ))
        # time.sleep(0.2)
    return results

# 진짜 쓰는 놈
def news_crawling(keyword):
    href_links = get_keyword_news(keyword)
    news_results = get_news_from_naver(keyword, href_links)
    return news_results

---
## 3. 그래프 요소 정의
### 1. 상태 작성

In [ ]:

State = {
    "keyword": str,                # 분석 대상 키워드
    "raw_documents": [SearchResult],   # 크롤링/뉴스 등 원문 문서들 (content, link, source 등)
    "embedded_documents": [dict],  # 임베딩 후 VDB에 저장된 문서 메타
    "retrieved_documents": [dict], # VDB에서 keyword 기반으로 가져온 후보 문서들
    "validated_documents": [dict], # 키워드와 실제로 관련 있다고 LLM이 yes 준 문서들
    "news_summaries": [str],       # 뉴스 개별 요약 결과들
    "trend_analysis": dict,        # 트렌드 원인 분석 (LLM 결과: answer + link[])
    "final_result": dict           # 최종 JSON (keyword, description, content, tags, category, refered)
}

### 2. 노드 구성

In [ ]:
# 데이터 수집 부분
def collect_sources(state):
    print(f"데이터 수집 중: {state["keyword"]}")
    # 1) keyword로 외부 소스 크롤링
    fetched_docs = news_crawling(state["keyword"])
    # 2) 결과를 [{"keyword": ..., "link": ..., "content": ...}, ...] 형태로 수집
    state["raw_documents"] = fetched_docs
    return state

# 데이터 임베딩
def embed_and_store(state):
    print("수집한 문서 임베딩 중")
    # 문서별 embedding 계산 -> VDB에 저장
    docs = embed_documents(state["raw_documents"])
    # 저장 후, vector_id 등 메타 정리
    # 저장 결과 출력
    state["embedded_documents"] = docs
    return state

# VDB에서 Retrieve
def retrieve_from_vdb(state):
    print("데이터 검색 중")
    retrieved = search_documents(state["keyword"])
    state["retrieved_documents"] = retrieved  # 예: [{"content":..., "link":..., ...}, ...]
    return state

# 각 문서 검증
def validate_relevance(state):
    print("각 문서 검증")
    validated = []
    for doc in state["retrieved_documents"]:
        judgment = llm_validate(keyword=state["keyword"], doc=doc["content"])
        if judgment["validation"] == "yes":
            validated.append({**doc, "reason": judgment["reason"]})
    state["validated_documents"] = validated
    return state

# 원문 요약
def summarize_news_individual(state):
    print("원문 요약 중")
    summaries = []
    for doc in state["raw_documents"]:
        summary = llm_main(doc["content"])
        summaries.append({
            "link": doc["link"],
            "summary": summary
        })
    state["news_summaries"] = summaries
    return state

# 분석 및 이유 작성
def analyze_trend_reason(state):
    print("최종 분석 중")
    trend_json = llm_main(
        keyword=state["keyword"],
        docs=state["validated_documents"],
        summaries=state["news_summaries"]
    )
    state["trend_analysis"] = trend_json  # {"answer": "...", "link": ["...", "..."]}
    return state

# 먼저 정의된 카테고리 목록
ALLOWED_CATEGORIES = [
    "건강", "게임", "과학", "기술", "기타", "기후",
    "미용 및 패션", "법률 및 정부", "반려동물 및 동물",
    "비즈니스 및 금융", "쇼핑", "스포츠", "식음료",
    "엔터테이먼트", "자동차", "정치", "취업 및 교육"
]

# 태그, 카테고리 붙이기
def classify_and_package(state):
    print("태그, 카테고리 붙이는 중 ")
    packaged = llm_main(
        keyword=state["keyword"],
        trend_answer=state["trend_analysis"]["answer"],
        links=state["trend_analysis"]["link"],
        allowed_categories=ALLOWED_CATEGORIES,
    )
    state["final_result"] = packaged
    return state


### 3. 노드 잇기 (그래프 짜기)

In [11]:
from langgraph.graph import StateGraph

workflow = StateGraph(dict)  # dict 대신 위에서 정의한 State 모델 사용 권장

workflow.add_node("collect_sources", collect_sources)
workflow.add_node("embed_and_store", embed_and_store)
workflow.add_node("retrieve_from_vdb", retrieve_from_vdb)
workflow.add_node("validate_relevance", validate_relevance)
workflow.add_node("summarize_news_individual", summarize_news_individual)
workflow.add_node("analyze_trend_reason", analyze_trend_reason)
workflow.add_node("classify_and_package", classify_and_package)

workflow.add_edge("collect_sources", "embed_and_store")
workflow.add_edge("embed_and_store", "retrieve_from_vdb")
workflow.add_edge("retrieve_from_vdb", "validate_relevance")
workflow.add_edge("validate_relevance", "summarize_news_individual")
workflow.add_edge("summarize_news_individual", "analyze_trend_reason")
workflow.add_edge("analyze_trend_reason", "classify_and_package")

workflow.set_entry_point("collect_sources")
workflow.set_finish_point("classify_and_package")

app = workflow.compile()

In [12]:
initial_state = {"keyword":"금융투자협회"}
app.invoke(initial_state)

데이터 수집 중: 금융투자협회
수집한 문서 임베딩 중
데이터 검색 중
각 문서 검증


TypeError: 'Document' object is not subscriptable